In [1]:
import os
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.utils import to_categorical
import cv2
import tkinter as tk
from tkinter import filedialog
from PIL import ImageTk, Image

2024-11-05 23:05:14.653560: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
from keras.layers import Activation
IMG_SIZE = 64

In [7]:
# モデルの作成とトレーニング
data = []
labels = []
classes = ['cat', 'dog']

for c in classes:
    path = os.path.join(os.getcwd(), c)
    for img in os.listdir(path):
        try:
            img_path = os.path.join(path, img)
            image = cv2.imread(img_path)
            # ⭐️画像サイズを一律にする
            image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
            data.append(image)
            labels.append(classes.index(c))
            print(img_path)
        except Exception as e:
            print(e)

data = np.array(data)
labels = np.array(labels)

/Users/mizuno-hikaru/Desktop/notebook_test/cat/cat1.png
/Users/mizuno-hikaru/Desktop/notebook_test/cat/cat2.png
/Users/mizuno-hikaru/Desktop/notebook_test/cat/cat3.png
/Users/mizuno-hikaru/Desktop/notebook_test/cat/cat4.png
/Users/mizuno-hikaru/Desktop/notebook_test/cat/cat5.png
/Users/mizuno-hikaru/Desktop/notebook_test/dog/dog5.png
/Users/mizuno-hikaru/Desktop/notebook_test/dog/dog4.png
/Users/mizuno-hikaru/Desktop/notebook_test/dog/dog3.png
/Users/mizuno-hikaru/Desktop/notebook_test/dog/dog2.png
/Users/mizuno-hikaru/Desktop/notebook_test/dog/dog1.png


In [8]:
# データをシャッフルする
idx = np.arange(data.shape[0])
np.random.shuffle(idx)
data = data[idx]
labels = labels[idx]

In [9]:
# データをトレーニング用と検証用に分割する
num_samples = len(data)
num_train = int(num_samples * 0.8)
x_train = data[:num_train]
y_train = labels[:num_train]
x_val = data[num_train:]
y_val = labels[num_train:]

# 画像データの正規化
x_train = x_train / 255.0
x_val = x_val / 255.0

# ラベルをone-hotエンコーディングに変換する
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)

In [10]:
# モデルを構築する
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dense(2))
model.add(Activation('softmax'))

# モデルをコンパイルする
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# モデルをトレーニングする
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))

Epoch 1/10


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.6250 - loss: 0.6836 - val_accuracy: 0.5000 - val_loss: 0.7188
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - accuracy: 0.7500 - loss: 0.5696 - val_accuracy: 0.5000 - val_loss: 1.2107
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step - accuracy: 0.5000 - loss: 0.7390 - val_accuracy: 0.5000 - val_loss: 0.6961
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step - accuracy: 0.8750 - loss: 0.4302 - val_accuracy: 0.5000 - val_loss: 0.7304
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step - accuracy: 0.7500 - loss: 0.4439 - val_accuracy: 0.5000 - val_loss: 0.7227
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - accuracy: 0.8750 - loss: 0.3162 - val_accuracy: 0.5000 - val_loss: 0.8951
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step - accuracy: 0.8750 - loss: 0.2949 - val_accuracy: 0.5000 - val_loss: 0.8683
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - accuracy: 0.8750 - loss: 0.2412 - val_accuracy: 0.5000 - val_loss: 0.7268
Epoch 9/10
1/

In [11]:
# GUIの作成
root = tk.Tk()
root.title("Cat or Dog Classifier")
root.geometry("500x500")


# ボタンのクリック時の処理
def predict():
    # 画像を選択する
    file_path = filedialog.askopenfilename()

    # 選択された画像を表示する
    image_tk = Image.open(file_path).resize((IMG_SIZE, IMG_SIZE))
    image_tk = ImageTk.PhotoImage(image_tk)
    img_label.configure(image=image_tk)
    img_label.image = image_tk
    print(file_path)

    # 選択された画像をモデルに入力して予測結果を表示する
    image_selected = cv2.imread(file_path)
    image_selected = cv2.resize(image_selected, (IMG_SIZE, IMG_SIZE))
    image_selected = np.expand_dims(image_selected, axis=0)
    image_selected = image_selected / 255.0
    prediction = model.predict(image_selected)
    if np.argmax(prediction) == 0:
        result_label.configure(text="This is a cat!")
    else:
        result_label.configure(text="This is a dog!")


# 画像を表示するラベルの作成
img_label = tk.Label(root)
img_label.pack()

# 予測結果を表示するラベルの作成
result_label = tk.Label(root, font=("Helvetica", 18))
result_label.pack()

# ボタンの作成
button = tk.Button(root, text="Choose Image", command=predict)
button.pack()

root.mainloop()

/Users/mizuno-hikaru/Desktop/download.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
/Users/mizuno-hikaru/Desktop/download-1.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
/Users/mizuno-hikaru/Desktop/download-2.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
/Users/mizuno-hikaru/Desktop/download-3.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
/Users/mizuno-hikaru/Desktop/images.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
/Users/mizuno-hikaru/Desktop/download.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
/Users/mizuno-hikaru/Desktop/download.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step


: 

In [1]:
print("end")

end
